In [1]:
#pip install -r requirements.txt

Processing /private/tmp/jupyterlab--argon2-cffi-20201109-55879-fd1k0d/argon2-cffi-20.1.0
ERROR: Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: '/private/tmp/jupyterlab--argon2-cffi-20201109-55879-fd1k0d/argon2-cffi-20.1.0'

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install pyhton-igraph

ERROR: Could not find a version that satisfies the requirement pyhton-igraph
ERROR: No matching distribution found for pyhton-igraph
Note: you may need to restart the kernel to use updated packages.


In [7]:
from igraph import Graph
from igraph import ARPACKOptions
import math
import numpy as np
import pandas as pd
import networkx as nx
import community as louvain_community
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [8]:
codes = pd.read_csv('data/codes_over_5mil.csv', delimiter=' ')
df = pd.read_csv('data/dij_joint_civ.csv')
df['positive_Dij'] = [pow(math.e,d) for d in df['D_ij']]
df5 = df[df['i'].isin(codes['Code'])]
df5 = df5[df5['j'].isin(codes['Code'])]
codes.set_index('Code', inplace=True)

In [9]:
codes.head()

,Index,Country
Code,,
AE,1,United Arab Emirates
AO,2,Angola
AR,3,Argentina
AT,4,Austria
AU,5,Australia


In [10]:
print(len(df), len(df5)) # perfect

10667 3828


In [25]:
G = Graph()
G.add_vertices(list(set(df5['i'])) + ['ZW'])
for i, row in df5.iterrows():
    G.add_edge(row['i'], row['j'], weight=row['positive_Dij'])

partition = G.community_leading_eigenvector(clusters=4, weights='weight', arpack_options=ARPACKOptions(max_iter=10000))
#partition = partition.as_clustering()
groups = {}
for i, g in enumerate(partition.subgraphs()):
    for n in g.vs['name']:
        groups[n] = i



In [26]:
codes['le4'] = 0
for k, v in groups.items():
    codes.loc[k, 'le4'] = v

In [27]:
codes[codes['ml'] == 1]

,Index,Country,ml,fg,sg,le2,le3,le4
Code,,,,,,,,
BD,7,Bangladesh,1,0,5,1,1,1
CA,15,Canada,1,1,7,0,0,3
CZ,23,Czech Republic,1,1,5,0,2,2
DO,26,Dominican Republic,1,1,5,1,1,1
GR,36,Greece,1,1,7,0,0,3
IN,43,India,1,1,7,0,2,2
MZ,58,Mozambique,1,1,7,0,0,3
PY,69,Paraguay,1,1,5,0,0,0
TH,79,Thailand,1,0,5,1,1,1


In [11]:
G = Graph()
G.add_vertices(list(set(df5['i'])) + ['ZW'])
for i, row in df5.iterrows():
    G.add_edge(row['i'], row['j'], weight=row['positive_Dij'])

partition = G.community_leading_eigenvector(clusters=3, weights='weight') # arpack_options=ARPACKOptions(max_iter=10000))
groups = {}
for i, g in enumerate(partition.subgraphs()):
    for n in g.vs['name']:
        groups[n] = i

In [12]:
codes['le3'] = 0
for k, v in groups.items():
    codes.loc[k, 'le3'] = v

In [13]:
codes[codes['le3'] == 0]

,Index,Country,le4,le3
Code,,,,
AO,2,Angola,3,0
AR,3,Argentina,0,0
AT,4,Austria,0,0
BI,11,Burundi,0,0
BO,12,Bolivia,3,0
CD,16,The Democratic Republic Of Congo,0,0
CH,17,Switzerland,0,0
CI,18,Cote D'Ivoire,0,0
CL,19,Chile,0,0


In [11]:
df5[(df5['i'] == 'AO') & (df5['j'] == 'CN')]

,i,j,D_ij,civ_i,civ_j,civ_ij,comciv,positive_Dij
460,AO,CN,-11.24013,afric,sinic,NaN,0,0.000013


In [12]:
df5[(df5['i'] == 'CH') & (df5['j'] == 'CN')]

,i,j,D_ij,civ_i,civ_j,civ_ij,comciv,positive_Dij
3579,CH,CN,0.022875,wstrn,sinic,NaN,0,1.023139


In [13]:
df5[(df5['i'] == 'ES') & (df5['j'] == 'FR')]

,i,j,D_ij,civ_i,civ_j,civ_ij,comciv,positive_Dij
5360,ES,FR,1.063779,wstrn,wstrn,wstrn,1,2.897299


In [28]:
#codes.to_csv('codesmappings.csv')

In [31]:
#codes = pd.read_csv('codesmappings.csv')
codes.set_index('Index', inplace=True, drop=False)

In [32]:
codes.head(2)

,Code,Index,Country,ml,fg,sg,le2,le3,le4
Index,,,,,,,,,
1,AE,1,United Arab Emirates,7,7,1,0,0,3
2,AO,2,Angola,5,5,3,0,2,2


In [46]:
og_groups = []
for code in codes['Code']:
    try:
        og_groups.append(str(list(set(df5[df5['i'] == code]['civ_i']))[0]))
    except:
        og_groups.append('NaN')

In [47]:
codes['groups'] = [i if i != 'nan' else 'NaN' for i in og_groups]

In [48]:
codes.head(20)

,Code,Index,Country,ml,fg,sg,le2,le3,le4,groups,group_num
Index,,,,,,,,,,,
1,AE,1,United Arab Emirates,7,7,1,0,0,3,islam,0
2,AO,2,Angola,5,5,3,0,2,2,afric,0
3,AR,3,Argentina,2,2,6,0,2,2,latam,0
4,AT,4,Austria,2,2,6,0,2,2,wstrn,0
5,AU,5,Australia,6,6,10,0,0,3,wstrn,0
6,AZ,6,Azerbaijan,2,2,6,1,1,1,islam,0
7,BD,7,Bangladesh,1,0,5,1,1,1,islam,0
8,BE,8,Belgium,5,9,10,0,0,3,wstrn,0
9,BF,9,Burkina Faso,0,0,1,0,0,0,afric,0


In [49]:
group_map = {'islam':0, 'afric':1, 'latam':2, 'wstrn':3, 'ortho':4, 'NaN':5, 'buddh':6, 'sinic':7, 'hindu':8}

In [50]:
set(codes['groups'])

{'NaN', 'afric', 'buddh', 'hindu', 'islam', 'latam', 'ortho', 'sinic', 'wstrn'}

In [51]:
codes['group_num'] = 0
for i, row in codes.iterrows():
    print(row['groups'])
    codes.loc[i, 'group_num'] = group_map[codes.loc[i, 'groups']] 

islam
afric
latam
wstrn
wstrn
islam
islam
wstrn
afric
ortho
afric
latam
latam
ortho
wstrn
NaN
wstrn
afric
latam
afric
sinic
latam
wstrn
wstrn
wstrn
latam
islam
latam
islam
wstrn
NaN
wstrn
wstrn
wstrn
afric
ortho
latam
latam
NaN
wstrn
islam
wstrn
hindu
wstrn
islam
NaN
afric
buddh
sinic
ortho
buddh
buddh
islam
afric
islam
latam
islam
afric
afric
latam
wstrn
hindu
latam
wstrn
wstrn
islam
wstrn
wstrn
latam
NaN
ortho
islam
islam
wstrn
buddh
wstrn
islam
latam
buddh
islam
islam
ortho
wstrn
islam
latam
sinic
islam
afric
afric
NaN


In [52]:
codes.head()

,Code,Index,Country,ml,fg,sg,le2,le3,le4,groups,group_num
Index,,,,,,,,,,,
1,AE,1,United Arab Emirates,7,7,1,0,0,3,islam,0
2,AO,2,Angola,5,5,3,0,2,2,afric,1
3,AR,3,Argentina,2,2,6,0,2,2,latam,2
4,AT,4,Austria,2,2,6,0,2,2,wstrn,3
5,AU,5,Australia,6,6,10,0,0,3,wstrn,3


In [53]:
codes.to_csv('data/community_detection_encodings.csv')